In [1]:
from sympy import *
import numpy as np 
init_printing()

comp = symbols( 'comp' )

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44 = symbols('D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44')
Exx, Ezz, Gxz, divV = symbols('Exx, Ezz, Gxz, divV')
stress_labels = ['Txx', 'Tzz', 'Txz', 'P']

# Dv = Matrix( [ [D11, 0, 0, 0], [0, D22, 0, 0], [0, 0, D33, 0], [0, 0, 0, 0]  ] )
Dv = Matrix( [ [D11, D12, D13, D14], [D21, D22, D23, D24], [D31, D32, D33, D34], [D41, D42, D43, D44]  ] )
Ed = Matrix( [ [Exx], [Ezz], [Gxz], [divV] ] )
T  = Dv*Ed;
for i in range(4):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )

Txx = D11*Exx + D12*Ezz + D13*Gxz + D14*divV;
Tzz = D21*Exx + D22*Ezz + D23*Gxz + D24*divV;
Txz = D31*Exx + D32*Ezz + D33*Gxz + D34*divV;
P = D41*Exx + D42*Ezz + D43*Gxz + D44*divV;


In [13]:
# Compressibility switch
comp = symbols('comp')

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')
dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension 1
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')
dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pW, pE, pSW, pSE, pNW, pNE = symbols('pW, pE, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pW, pE, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)
out_of_plane                        = symbols('out_of_plane')

inW,inE,inS,inN         = symbols('inW,inE,inS,inN')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
wW,wE,wS,wN             = symbols('wW,wE,wS,wN')
inSWW, inNWW            = symbols('inSWW, inNWW')
inSEE, inNEE            = symbols('inSEE, inNEE')

# Divergences
divW      = out_of_plane*( (uC  - uW )/dx + (vNW - vSW )/dz ) 
divE      = out_of_plane*( (uE  - uC )/dx + (vNE - vSE )/dz )
divSW     = out_of_plane*( (uS  - uSW)/dx + (vSW - vSSW)/dz )
divSE     = out_of_plane*( (uSE - uS )/dx + (vSE - vSSE)/dz )
divNW     = out_of_plane*( (uN  - uNW)/dx + (vNNW- vNW )/dz )
divNE     = out_of_plane*( (uNE - uN )/dx + (vNNE- vNE )/dz )
# divW      = (uC  - uW )/dx + (vNW - vSW )/dz 
# divE      = (uE  - uC )/dx + (vNE - vSE )/dz
# divSW     = (uS  - uSW)/dx + (vSW - vSSW)/dz
# divSE     = (uSE - uS )/dx + (vSE - vSSE)/dz
# divNW     = (uN  - uNW)/dx + (vNNW- vNW )/dz
# divNE     = (uNE - uN )/dx + (vNNE- vNE )/dz
# Deviatoric normal strain
ExxW      = ((uC -uW )/dx - comp*1/3*divW) 
EzzW      = ((vNW-vSW)/dz - comp*1/3*divW)
ExxE      = ((uE -uC )/dx - comp*1/3*divE)
EzzE      = ((vNE-vSE)/dz - comp*1/3*divE)
# Shear strain
# GxzN      = (inN*(uN - uC )/dz + (vNE -vNW )/dx) 
# GxzS      = (inS*(uC - uS )/dz + (vSE -vSW )/dx)
# Additional missing stress tensor components via interpolation
GxzNE     = (inN*(uNE - uE )/dz + inNEE*( vNEE - vNE  )/dx)
GxzSE     = (inS*(uE  - uSE)/dz + inSEE*( vSEE - vSE  )/dx)
GxzNW     = (inN*(uNW - uW )/dz + inNWW*( vNW  - vNWW )/dx)
GxzSW     = (inS*(uW  - uSW)/dz + inSWW*( vSW  - vSWW )/dx)

# # # Shear strain rate
inNv, inSv     = symbols('inNv, inSv') 
GxzN      = inNv* (inN*(uN - uC )/dz + (vNE -vNW )/dx)        # weird mixture 29/12/21
GxzS      = inSv* (inS*(uC - uS )/dz + (vSE -vSW )/dx) 
# # Additional missing stress tensor components via interpolation
# GxzNE     = inNEv* ( inN*(uNE - uE )/dz + inNEE*( vNEE - vNE  )/dx) # There could be an issue with inN instead of inNE
# GxzSE     = inSEv* ( inS*(uE  - uSE)/dz + inSEE*( vSEE - vSE  )/dx) # ...
# GxzNW     = inNWv* ( inN*(uNW - uW )/dz + inNWW*( vNW  - vNWW )/dx) # ...
# GxzSW     = inSWv* ( inS*(uW  - uSW)/dz + inSWW*( vSW  - vSWW )/dx) # ...

ExxNE     = ((uNE -uN )/dx - comp*1/3*divNE)
ExxNW     = ((uN  -uNW)/dx - comp*1/3*divNW)
ExxSE     = ((uSE -uS )/dx - comp*1/3*divSE)
ExxSW     = ((uS  -uSW)/dx - comp*1/3*divSW)
EzzNE     = ((vNNE-vNE)/dz - comp*1/3*divNE)
EzzNW     = ((vNNW-vNW)/dz - comp*1/3*divNW)
EzzSE     = ((vSE-vSSE)/dz - comp*1/3*divSE)
EzzSW     = ((vSW-vSSW)/dz - comp*1/3*divSW)
# INV 0
# GxzE      = wE*( inN*GxzN + inS*GxzS + inNEv*GxzNE + inSEv*GxzSE)
# GxzW      = wW*( inN*GxzN + inS*GxzS + inNWv*GxzNW + inSWv*GxzSW)
# GxzE      =  0.25*( GxzN + GxzS + GxzNE + GxzSE)                       # important update important to check wether it works with a FS
# GxzW      =  0.25*( GxzN + GxzS + GxzNW + GxzSW)
GxzE      = 0.25*( inN*GxzN + inS*GxzS + inNEv*GxzNE + inSEv*GxzSE)      # weird mixture 29/12/21
GxzW      = 0.25*( inN*GxzN + inS*GxzS + inNWv*GxzNW + inSWv*GxzSW)
ExxN      = wN*( inE*ExxE + inW*ExxW + inNWc*ExxNW + inNEc*ExxNE)
ExxS      = wS*( inE*ExxE + inW*ExxW + inSWc*ExxSW + inSEc*ExxSE)
EzzN      = wN*( inE*EzzE + inW*EzzW + inNWc*EzzNW + inNEc*EzzNE) 
EzzS      = wS*( inE*EzzE + inW*EzzW + inSWc*EzzSW + inSEc*EzzSE)

pS        = wS*( inE*pE + inW*pW + inSWc*pSW + inSEc*pSE)
pN        = wN*( inE*pE + inW*pW + inNWc*pNW + inNEc*pNE)  

#----------------------------------------#
# ExxW = 1/dx*( uC- uW) - comp*1/3*divW
# EzzW = 1/dz*(vNW-vSW) - comp*1/3*divW
# GxzW = 1/4*(GxzS+GxzN+GxzSW+GxzNW)
EdW  = Matrix([[inW*ExxW], [inW*EzzW], [GxzW], [inW*pW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W', D14:'D14W'})
#----------------------------------------#
# ExxE = 1/dx*( uE- uC) - comp*1/3*divE
# EzzE = 1/dz*(vNE-vSE) - comp*1/3*divE
# GxzE = 1/4*(GxzS+GxzN+GxzSE+GxzNE)
EdE  = Matrix([[inE*ExxE], [inE*EzzE], [GxzE], [inE*pE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E', D14:'D14E'})
#----------------------------------------#
# ExxS = 1/4*(ExxE+ExxW+ExxSE+ExxSW) - comp*1/3*divS
# EzzS = 1/4*(EzzE+EzzW+EzzSE+EzzSW) - comp*1/3*divS
# GxzS = 1/dz*(uC-uS) + 1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [inS*GxzS], [inS*pS]])
# EdS  = Matrix([[inS*ExxS], [inS*EzzS], [inS*GxzS], [inS*pS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S', D34:'D34S'})
#----------------------------------------#
# ExxN = 1/4*(ExxE+ExxW+ExxNE+ExxNW) - comp*1/3*divN
# EzzN = 1/4*(EzzE+EzzW+EzzNE+EzzNW) - comp*1/3*divN
# GxzN = 1/dz*(uN-uC) + 1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [inN*GxzN], [inN*pN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N', D34:'D34N'})
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS) 
Fx = -Fx
for i in range(len(dofs)):
# for i in range(6):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cUc)) + ';')

uW = (1.0/3.0)*(-0.75*D13W*dx*(inN*inNWv - inS*inSWv) + dx*inW*(-D31N*wN*(comp*out_of_plane - 3) + D31S*wS*(comp*out_of_plane - 3) - D32N*comp*out_of_plane*wN + D32S*comp*out_of_plane*wS) + dz*inW*(D11W*(comp*out_of_plane - 3) + D12W*comp*out_of_plane))/(pow(dx, 2)*dz);
uC = (1.0/3.0)*(dx*(3*dx*(D33N*pow(inN, 2)*inNv + D33S*pow(inS, 2)*inSv) + dz*(inE - inW)*(-D31N*wN*(comp*out_of_plane - 3) + D31S*wS*(comp*out_of_plane - 3) - D32N*comp*out_of_plane*wN + D32S*comp*out_of_plane*wS)) - dz*(0.75*dx*(-D13E + D13W)*(pow(inN, 2)*inNv - pow(inS, 2)*inSv) + dz*(D11E*inE*(comp*out_of_plane - 3) + D11W*inW*(comp*out_of_plane - 3) + D12E*comp*inE*out_of_plane + D12W*comp*inW*out_of_plane)))/(pow(dx, 2)*pow(dz, 2));
uE = (1.0/3.0)*(0.75*D13E*dx*(inN*inNEv - inS*inSEv) + dx*inE*(D31N*wN*(comp*out_of_plane - 3) - D31S*wS*(comp*out_of_plane - 3) + D32N*comp*out_of_plane*wN - D32S*comp*out_of_plane*wS) + dz*inE*(D11E*(comp*out_of_plane - 3) + D12E*comp*out_of_plane))/(pow(dx, 2)*dz);
uS = (-D33S*dx*po

In [14]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])

# Stencil extension 1
vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE = symbols('vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE')
dofs_ext = Matrix([vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pS, pN, pSW, pSE, pNW, pNE = symbols('pS, pN, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pS, pN, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
inNNW, inNNE = symbols('inNNW, inNNE')
inSSW, inSSE = symbols('inSSW, inSSE')

wS,wN,wW,wE = symbols('wS,wN,wW,wE ')

# Divergences
divS      = out_of_plane*( (uSE-uSW)/dx + (vC -vS )/dz )   
divN      = out_of_plane*( (uNE-uNW)/dx + (vN -vC )/dz )  
divSW     = out_of_plane*( (uSW-uSWW)/dx + (vW -vSW )/dz )
divSE     = out_of_plane*( (uSEE-uSE)/dx + (vE -vSE )/dz )
divNW     = out_of_plane*( (uNW-uNWW)/dx + (vNW -vW )/dz )
divNE     = out_of_plane*( (uNEE-uNE)/dx + (vNE -vE )/dz ) 
# Deviatoric normal strain
ExxS      = ((uSE-uSW)/dx - comp*1/3*divS)
EzzS      = ((vC -vS )/dz - comp*1/3*divS)
ExxN      = ((uNE-uNW)/dx - comp*1/3*divN)
EzzN      = ((vN -vC )/dz - comp*1/3*divN)
# Shear strain
# GxzE      = ((uNE-uSE)/dz + inE*(vE - vC )/dx)
# GxzW      = ((uNW-uSW)/dz + inW*(vC - vW )/dx)
# Additional missing stress tensor components via interpolation
GxzSW     = (inSSW*(uSW  - uSSW)/dz + inW*(vS  - vSW)/dx)
GxzSE     = (inSSE*(uSE  - uSSE)/dz + inE*(vSE - vS )/dx)
GxzNW     = (inNNW*(uNNW - uNW )/dz + inW*(vN  - vNW)/dx)
GxzNE     = (inNNE*(uNNE - uNE )/dz + inE*(vNE - vN )/dx)

# include FS
inEv, inWv = symbols('inEv, inWv')
# Shear strain
GxzE      = inEv*((uNE-uSE)/dz + inE*(vE - vC )/dx)  # weird mixture 29/12/21
GxzW      = inWv*((uNW-uSW)/dz + inW*(vC - vW )/dx)
# # Additional missing stress tensor components via interpolation
# GxzSW     = inSWv*(inSSW*(uSW  - uSSW)/dz + inW*(vS  - vSW)/dx)
# GxzSE     = inSEv*(inSSE*(uSE  - uSSE)/dz + inE*(vSE - vS )/dx)
# GxzNW     = inNWv*(inNNW*(uNNW - uNW )/dz + inW*(vN  - vNW)/dx)
# GxzNE     = inNEv*(inNNE*(uNNE - uNE )/dz + inE*(vNE - vN )/dx)

ExxNE     = ((uNEE-uNE)/dx - comp*1/3*(divNE))
ExxNW     = ((uNW-uNWW)/dx - comp*1/3*(divNW))
ExxSE     = ((uSEE-uSE)/dx - comp*1/3*(divSE))
ExxSW     = ((uSW-uSWW)/dx - comp*1/3*(divSW))
EzzNE     = ((vNE -vE )/dz - comp*1/3*(divNE))
EzzNW     = ((vNW -vW )/dz - comp*1/3*(divNW))
EzzSE     = ((vE -vSE )/dz - comp*1/3*(divSE))
EzzSW     = ((vW -vSW )/dz - comp*1/3*(divSW))

# INV 0
# GxzN      = wN*( inW*GxzW + inE*GxzE + inNWv*GxzNW + inNEv*GxzNE )
# GxzS      = wS*( inW*GxzW + inE*GxzE + inSWv*GxzSW + inSEv*GxzSE )
# GxzN      = 0.25*( GxzW + GxzE + GxzNW + GxzNE )
# GxzS      = 0.25*( GxzW + GxzE + GxzSW + GxzSE )
GxzN      = 0.25*( inW*GxzW + inE*GxzE + inNWv*GxzNW + inNEv*GxzNE )
GxzS      = 0.25*( inW*GxzW + inE*GxzE + inSWv*GxzSW + inSEv*GxzSE ) # weird mixture 29/12/21
ExxE      = wE*( inS*ExxS + inN*ExxN + inNEc*ExxNE + inSEc*ExxSE )
ExxW      = wW*( inS*ExxS + inN*ExxN + inNWc*ExxNW + inSWc*ExxSW )
EzzE      = wE*( inS*EzzS + inN*EzzN + inNEc*EzzNE + inSEc*EzzSE )
EzzW      = wW*( inS*EzzS + inN*EzzN + inNWc*EzzNW + inSWc*EzzSW )

pE      = wE*( inS*pS + inN*pN + inNEc*pNE + inSEc*pSE )
pW      = wW*( inS*pS + inN*pN + inNWc*pNW + inSWc*pSW )

#----------------------------------------#
# ExxS = 1/dx*(uSE-uSW) - comp*1/3*divS
# EzzS = 1/dz*( vC- vS) - comp*1/3*divS
# GxzS = 1/4*(GxzW+GxzE+GxzSW+GxzSE)
EdS  = Matrix([[inS*ExxS], [inS*EzzS], [inS*GxzS], [inS*pS]])
TS   = Dv*EdS
TzzS = TS[1].subs({D21:'D21S', D22:'D22S', D23:'D23S', D24:'D24S'})
#----------------------------------------#
# ExxN = 1/dx*(uNE-uNW) - comp*1/3*divN
# EzzN = 1/dz*( vN- vC) - comp*1/3*divN
# GxzN = 1/4*(GxzW+GxzE+GxzNW+GxzNE)
EdN  = Matrix([[inN*ExxN], [inN*EzzN], [inN*GxzN], [inN*pN]])
TN   = Dv*EdN
TzzN = TN[1].subs({D21:'D21N', D22:'D22N', D23:'D23N', D24:'D24N'})
#----------------------------------------#
# ExxW = 1/4*(ExxS+ExxN+ExxSW+ExxNW) - comp*1/3*divW
# EzzW = 1/4*(EzzS+EzzN+EzzSW+EzzNW) - comp*1/3*divW
# GxzW = 1/dz*(uNW-uSW) + 1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [inW*GxzW], [inW*pW]])
TW   = Dv*EdW
TxzW = TW[2].subs({D31:'D31W', D32:'D32W', D33:'D33W', D34:'D34W'})
# #----------------------------------------#
# ExxE = 1/4*(ExxS+ExxN+ExxSE+ExxNE) - comp*1/3*divE
# EzzE = 1/4*(EzzS+EzzN+EzzSE+EzzNE) - comp*1/3*divE
# GxzE = 1/dz*(uNE-uSE) + 1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [inE*GxzE], [inE*pE]])
TE   = Dv*EdE
TxzE = TE[2].subs({D31:'D31E', D32:'D32E', D33:'D33E', D34:'D34E'})
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
Fz = -Fz
for i in range(len(dofs)):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cVc)) + ';') #print(cUc)

vW = (-D33W*dz*pow(inW, 2)*inWv + 0.25*dx*pow(inW, 2)*inWv*(D23N*inN - D23S*inS) + (1.0/3.0)*dx*wW*(inNWc - inSWc)*(D31W*comp*out_of_plane + D32W*(comp*out_of_plane - 3)))/(pow(dx, 2)*dz);
vC = (1.0/3.0)*(-dx*(dx*(D21N*comp*inN*out_of_plane + D21S*comp*inS*out_of_plane + D22N*inN*(comp*out_of_plane - 3) + D22S*inS*(comp*out_of_plane - 3)) + 0.75*dz*(-D23N*inN + D23S*inS)*(pow(inE, 2)*inEv - pow(inW, 2)*inWv)) + dz*(dx*(inN - inS)*(-D31E*comp*out_of_plane*wE + D31W*comp*out_of_plane*wW - D32E*wE*(comp*out_of_plane - 3) + D32W*wW*(comp*out_of_plane - 3)) + 3*dz*(D33E*pow(inE, 2)*inEv + D33W*pow(inW, 2)*inWv)))/(pow(dx, 2)*pow(dz, 2));
vE = (-D33E*dz*pow(inE, 2)*inEv + 0.25*dx*pow(inE, 2)*inEv*(-D23N*inN + D23S*inS) - 1.0/3.0*dx*wE*(inNEc - inSEc)*(D31E*comp*out_of_plane + D32E*(comp*out_of_plane - 3)))/(pow(dx, 2)*dz);
vS = (1.0/3.0)*inS*(-0.75*D23S*dz*(inE*inSEv - inSWv*inW) + dx*(D21S*comp*out_of_plane + D22S*(comp*out_of_plane - 3)) + dz*(-D31E*comp*out_of_plane*wE + D31W*comp*out_of_